In [68]:
from math import sqrt
from decimal import Decimal
from statistics import mean
import numpy as np
import random
import itertools
from itertools import zip_longest
from itertools import combinations  
from functools import reduce
from itertools import groupby
import pandas as pd #only used to read csv
    

#####################################################HELPER FUNCTIONS######################################################
def euclid_dist(x2,x1,y2,y1):
    """returns the distance between points (x1,y1) and (x2,y2)"""
    return sqrt(((x2-x1)**2) + ((y2-y1)**2))

def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx
    args = [iter(iterable)] * n
    return zip_longest(fillvalue=fillvalue, *args)

def rCombos(arr, r): 
    return list(combinations(arr, r)) 
##########################################################################################################################

# Kmeans 

In [69]:
def set_centroids(points, num_clusters):
    """pick random coordinates for k initial centroids; return a list of tuples"""
    X = [x[0] for x in points]       
    Y = [y[1] for y in points]
    
    #initialize k random centroids
    centroids = []
    for k in list(range(num_clusters)):
        X_centroid = random.randint(min(X), max(X))
        Y_centroid = random.randint(min(Y), max(Y))
        centroids.append((X_centroid, Y_centroid))
    print('INITIAL CENTROIDS: ' + str(centroids))
    print('\n')
    return centroids



def get_clusters(centroids,points):
    """determine the clusters to which each point belongs based on the current centroid locations"""
    #find the distance between every point and every centroid
    distances = []
    for centroid in centroids: #compute the distances from each centroid to each point
        for point in points:
            distances.append(euclid_dist(point[0],centroid[0],point[1],centroid[1]))
            #point[0] = x2, point[1] = y2, centroid[0] = x1, centroid[1] = y1

    #partition contiguous list of distances into per-cluster groups (i.e. group every len(points) items)
    bins = []
    partition_size = len(points) 
    for group in grouper(distances, partition_size):
        bins.append(group) #bins is a list of lists

    per_point_distances = list(zip(*bins)) #unpack arguments to zip together 'n' lists (this gets the k distances per point)
    #per_point_distances example data structure (k=3): 
    #[(2.1, 1.5, 3), (3.5, 4, 1.2),...] --> [point1(dist(C1), dist(C2), dist(C3)), point2(dist(C1), dist(C2), dist(C3)) . . .]

    #assign points to clusters based on min distances (nearest neighbor)
    cluster_assign = [] #list to store cluster assignments by index (0-n)
    for group in per_point_distances: #'group' represents the k distances per (x,y) pair
        _min = min(group)
        cluster_assign.append(group.index(_min)) #store the index of the group (0-k) which contains the min distance
    clusters = list(zip(points,cluster_assign))
    return clusters



def between_cluster_var(centroids):
    """takes the coordinates of k centroids as a list of tuples, and computes between cluster variation"""
    #get all combinations of centroid pairs get compute sum squared distance between them
    centroid_combos = rCombos(centroids,2) #every combination of pairs of 2
    bc_dist = [] #list to store between cluster distances
    for combo in centroid_combos:
        bc_dist.append(euclid_dist(combo[1][0],combo[0][0],combo[1][1],combo[0][1]))   
    sumsq=0
    for dist in bc_dist:
        sumsq += dist*dist
    #bcv = between cluster variation
    bcv = sumsq #between cluster variation is the sum of squared distances between all centroid combos
    print('BCV: ' + str(bcv))
    return bcv



def within_cluster_var(clusters, centroids):
    """takes a list of tuples of (x,y) pairs and cluster assignments, and a list of centroids, and computes WCV"""
    #within cluster variation = sum of squared deviations from cluster mean
    squared_dev = [] #list to store within cluster sum of squared deviations
    for item in clusters:
        squared_dev.append((euclid_dist(item[0][0], centroids[item[1]][0], item[0][1], centroids[item[1]][1])))
    sumsq=0
    for dev in squared_dev:
        sumsq += dev*dev
    #wcv = within cluster variation
    wcv = sumsq
    print('WCV: ' + str(wcv))
    return wcv



def update_centroids(clusters, points):
    #cluster assignments = points clusters ---> get the average x and y coordinate for points in each cluster
    assigns = []
    for item in clusters:
        assigns.append(item[1])
    cluster_assignments = list(zip(assigns,points))
    assign_dict = dict.fromkeys([x[0] for x in cluster_assignments])        
    for key, value in assign_dict.items():
        assign_dict[key] = [x[1] for x in cluster_assignments if x[0]==key] 

    #FIND THE AVERAGE COORDINATES OF CURRENT CENTROIDS TO CREATE NEW CENTROIDS
    for key,value in assign_dict.items():
        xs = []
        ys = []
        for point in value:
            xs.append(point[0])
            ys.append(point[1])
        avg_x = mean(xs)
        avg_y = mean(ys)
        new_centroid = (avg_x, avg_y)
        assign_dict[key] = new_centroid

    #set updated centroids variable for next iteration
    centroids = dict(sorted(assign_dict.items()))

    return centroids  



def kmeans(points, num_clusters, num_iterations):
    centroids = set_centroids(points, num_clusters) #set random centroids within the range of input data
    clusters = get_clusters(centroids, points) #get cluster assignments for each point
    bcv = between_cluster_var(centroids) #calculate the between cluster variance
    wcv = within_cluster_var(clusters,centroids) #calculate the within cluster variance
    ratio = bcv / wcv #this ratio should increase within each iteration
    print('INITIAL BCV / WCV RATIO: ' + str(ratio))

    for i in range(num_iterations):
        print('\n')
        print('------------------------------------------------------------------------------------------------------------------------------')
        centroids = list(update_centroids(clusters, points).values()) #update centroids to avg coordinates of member points
        print('NEW CENTROIDS: ' + str(centroids))
        clusters = get_clusters(centroids, points) #get new cluster assignments for each point
        bcv = between_cluster_var(centroids) #update bcv
        wcv = within_cluster_var(clusters, centroids) #update wcv (should decrease on every iteration)
        ratio = bcv / wcv 
        print('BCV / WCV ratio: ' + str(ratio))

    return clusters

In [77]:
if __name__ == "__main__": 
    points = pd.read_csv('kmeansData.csv', sep=',') #kmeansData.csv was generated randomly in Excel
    X = list(points.iloc[:,0])
    Y = list(points.iloc[:,1])
    points = list(zip(X,Y))

    cluster_membership = kmeans(points,num_clusters=6,num_iterations=10)

    print('\n')
    print('Snapshot of cluster membership (first 10 points): ' + str(cluster_membership[0:10]))

INITIAL CENTROIDS: [(10, -10), (-9, 1), (5, 5), (3, -2), (0, 3), (8, 9)]


BCV: 2669.0
WCV: 5620.0
INITIAL BCV / WCV RATIO: 0.47491103202846974


------------------------------------------------------------------------------------------------------------------------------
NEW CENTROIDS: [(6.857142857142857, -7.666666666666667), (-7.423076923076923, 0.28846153846153844), (5.357142857142857, 5.107142857142857), (2.673076923076923, -4.096153846153846), (-1.7647058823529411, 5), (7.25, 8.666666666666666)]
BCV: 2151.8882196021464
WCV: 4203.260423620662
BCV / WCV ratio: 0.5119569102854977


------------------------------------------------------------------------------------------------------------------------------
NEW CENTROIDS: [(7.34375, -6.125), (-7.341463414634147, -2.048780487804878), (5.482758620689655, 4.137931034482759), (1.048780487804878, -4.341463414634147), (-3.642857142857143, 6.476190476190476), (6.5, 8.857142857142858)]
BCV: 2218.9194902886024
WCV: 3065.3739330355756
BCV / WC